# Import the Libraries

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os

In [ ]:
import tensorflow as tf

# Loading the Model Weights and Architecture

In [ ]:
from tensorflow.keras.models import model_from_json

json_file = open('model_arch.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('model_weights.h5')

In [ ]:
face_recogniser = model

# Creating the Dataset of Embeddings

In [ ]:
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size = (160, 160))
    img = np.around(np.array(img) / 255.0, decimals = 12)
    x_train = np.expand_dims(img, axis = 0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord = 2)

In [ ]:
os.chdir("cricketer")

In [ ]:
database = {}
for i in os.listdir():
    if i != ".DS_Store":
        j = i.split(".")[0].title()
        database[j] = img_to_encoding(i, model)

# Input - Face Detection + Look-Alike

In [ ]:
def who_is_it(image_path, database, model):
    encoding = img_to_encoding(image_path, model)
    min_dist = 1e6
    
    for (name, enc) in database.items():
        dist = np.linalg.norm(encoding - enc)
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("It's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [ ]:
img = cv.imread("") # Enter image path 
gray_image = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

In [ ]:
face_classifier = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")

face = face_classifier.detectMultiScale(gray_image, scaleFactor = 1.1, minNeighbors = 5, minSize = (40, 40))

for (x, y, w, h) in face:
    cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)
    
faces = img[y + 3 : y + h - 2, x + 3 : x + w - 2]
img_rgb = cv.cvtColor(faces, cv.COLOR_BGR2RGB)

plt.figure(figsize = (20,10))
plt.imshow(img_rgb)
plt.axis('off')
plt.show()

In [ ]:
path = "face.png"
cv.imwrite(path, img_rgb)
who_is_it(path, database, face_recogniser)
os.remove("face.png")